# Adversarial Training

Adversarial Training of Robust CNN

## Imports

### GPU Information

The following GPU is used while Training the Model. Google Colab is used for Training so the GPU might change while running it later.

**GPU-Specifications:**
*   Name: Tesla P100-PCIE
*   GPU Memory: 16280MiB

In [ ]:
!nvidia-smi

### Importing Data
Files are uploaded in Google Drive. Notebook is connected to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Modulation-Classification/RadioML-Dataset/Data/CW-Attack

### Importing Libraries

Cleverhans is used for generating Adversarial Examples.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import seaborn as sns
import scipy.io
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

# Tensorflow Libraries
!pip3 install tensorflow==2.2.0
!pip3 install keras-tuner==1.0.2
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import Dense, Flatten, ReLU, Conv2D, Input, MaxPooling2D, BatchNormalization, AveragePooling2D, Reshape, ZeroPadding2D, Dropout, Activation, Permute, Concatenate
from tensorflow.keras.models import Model,Sequential

# ART Libraries
"""
!pip3 install adversarial-robustness-toolbox[tensorflow]
from art.attacks.evasion import CarliniL2Method
from art.estimators.classification import KerasClassifier
"""

# Cleverhans Libraries
!pip3 install cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.tf2.attacks.carlini_wagner_l2 import CarliniWagnerL2

### Importing Datasets

Importing Data

In [ ]:
# Loading Training Data
Clean_X_Train = np.load("../Clean-Data/Clean_X_Train.npy")
Clean_y_Train = np.load("../Clean-Data/Clean_y_Train.npy")

# Loading Validation Data
f = open("../Clean-Data/Clean_X_Valid.pkl","rb")
Clean_X_Valid = pickle.load(f)

f = open("../Clean-Data/Clean_y_Valid.pkl","rb")
Clean_y_Valid = pickle.load(f)

Processing Data

In [ ]:
Clean_X_Train = np.expand_dims(np.transpose(Clean_X_Train, (0,2,1)),axis=-1)

for snr in Clean_X_Valid.keys():
    Clean_X_Valid[snr] = np.expand_dims(np.transpose(Clean_X_Valid[snr], (0,2,1)),axis=-1)

## Designing Attack

**CW Attack**
```
CW = CarliniWagnerL2(
            model_fn = Robust_CNN,
            y = np.repeat(Target_Encoding.reshape(-1,len(Target_Encoding)),Size,axis=0),
            targeted = True,
            batch_size = Size,
            clip_max = Max,
            clip_min = Min,
            max_iterations = nb_Iterations,
            learning_rate = Learning_Rate
        )

X_Batch_Adv = CW.attack(X_Batch)
```

Modulation Schemes

In [ ]:
f = open("../Dataset/ModulationMap.pkl", "rb")
ModulationMap = pickle.load(f)
ModulationMap = dict((v,k) for k,v in ModulationMap.items())
ModulationMap

Attack Settings




In [ ]:
# 8PSK is considered as target
TargetLabel = '8PSK'
Target = ModulationMap[TargetLabel]
Target_Encoding = np.zeros((11,))
Target_Encoding[Target] = 1
print ("Target Encoding:", Target_Encoding)

# Parameters of CW
nb_Iterations = 100
Confidence = 0.9
Batch_Size = 1024
Learning_Rate = 0.001
Max = np.max(Clean_X_Train)
Min = np.min(Clean_X_Train)

## Model

Robust_CNN Model is designed and Adversally Trained and Examples generated from it are used to perform Transfer-Based Attack

### Creating Robust_CNN




In [ ]:
tf.keras.backend.clear_session()

Inp = Input(shape=(128,2,1))

x = Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='glorot_uniform')(Inp)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,1), padding='valid')(x)
x = Dropout(0.3)(x)

x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='glorot_uniform')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,1), padding='valid')(x)
x = Dropout(0.3)(x)

x = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='glorot_uniform')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,1), padding='valid')(x)
x = Dropout(0.3)(x)

x = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='glorot_uniform')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,1), padding='valid')(x)
x = Dropout(0.3)(x)

x = Flatten()(x)
x = Dense(128, activation='relu', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
Out = Dense(11,activation='softmax', kernel_initializer='he_normal')(x)

Robust_CNN = Model(Inp,Out)
Robust_CNN.summary()
tf.keras.utils.plot_model(Robust_CNN, to_file='Robust_CNN.png', show_shapes=True,show_layer_names=True)

### Training the Model

Model Training Settings

In [ ]:
LossFunction = tf.losses.CategoricalCrossentropy()
AccuracyFunction = tf.keras.metrics.Accuracy()
Optimizer = tf.optimizers.Adam()
Epochs = 250
Batch_Size = 3072
N = Clean_X_Train.shape[0]
Steps = int(np.ceil(N/Batch_Size))

Robust_CNN.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
History = {"Loss": np.zeros(Epochs), "Accuracy": np.zeros(Epochs)}

Train Step

In [ ]:
@tf.function
def train_step(x,y):
    with tf.GradientTape() as tape:
        # Predict
        y_pred = Robust_CNN(x, training=True)

        # Loss
        loss = LossFunction(y, y_pred)

    # Back Propagation
    gradients = tape.gradient(loss, Robust_CNN.trainable_variables)
    Optimizer.apply_gradients(zip(gradients, Robust_CNN.trainable_variables))

    return loss

Training Dataset

In [ ]:
Adv_X = np.copy(Clean_X_Train)
Adv_y = np.copy(Clean_y_Train)

Adversarial Training

In [ ]:
for epoch in range(Epochs):
    print ("Epoch-" + str(epoch) + ":")

    # Shuffling Data
    ShuffleIndex = np.random.choice(N, size=N, replace=False)
    Adv_X = np.copy(Clean_X_Train)
    
    # Adversarial Samples
    for step in range(Steps):
        i = step*Batch_Size
        j = min((step + 1)*Batch_Size, N)
        Indices = ShuffleIndex[i:j]

        X_Batch = np.copy(Clean_X_Train[Indices])
        Size = X_Batch.shape[0]

        # Adversarial Data
        CW = CarliniWagnerL2(
            model_fn = Robust_CNN,
            y = np.repeat(Target_Encoding.reshape(-1,len(Target_Encoding)),Size,axis=0),
            targeted = True,
            batch_size = Batch_Size,
            clip_max = Max,
            clip_min = Min,
            max_iterations = nb_Iterations,
            confidence = Confidence,
            learning_rate = Learning_Rate
        )

        X_Batch_Adv = CW.attack(X_Batch)
        
        Adv_X[Indices] = np.copy(X_Batch_Adv)
    
    X = np.append(Adv_X, Clean_X_Train, axis=0)
    y = np.append(Adv_y, Clean_y_Train, axis=0)
    # Train the Model
    TrainHistory = Robust_CNN.fit(X, y, epochs=1, batch_size=Batch_Size, verbose=1, shuffle=True)

    # Loss and Accuracy
    Loss = np.mean(TrainHistory.history['loss'])
    Acc = np.mean(TrainHistory.history['accuracy'])
    
    # Updating Loss
    History['Loss'][epoch] = Loss
    History['Accuracy'][epoch] = Acc

Plotting Loss

In [ ]:
plt.figure()

plt.subplot(1,2,1)
plt.grid()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss")
plt.plot(np.arange(Epochs), History['Loss'], label="Loss")
plt.legend()

plt.subplot(1,2,2)
plt.grid()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy")
plt.plot(np.arange(Epochs), History['Accuracy'], label="Accuracy")
plt.legend()

plt.show()

Save Model

In [ ]:
Robust_CNN.save("Robust_CNN")

### Evaluating Data

In [ ]:
def EvaluateData(Model, X_Valid, y_Valid, SavePath, ValidBatchSize=32):
    Valid_SNR = np.arange(-20,20,2)
    Accuracy = []

    print ("Evaluating Model")
    for snr in Valid_SNR:
        Pred_Labels = np.argmax(Model.predict(X_Valid[snr]),axis=1)
        True_Labels = np.argmax(y_Valid[snr],axis=1)

        Acc = accuracy_score(True_Labels, Pred_Labels)

        print ("SNR:", snr, "Accuracy:", Acc)
        Accuracy.append(Acc)

    Accuracy = np.array(Accuracy)

    plt.figure()
    plt.plot(Valid_SNR,Accuracy, color='blue')
    plt.scatter(Valid_SNR,Accuracy, color='red')
    plt.title("Accuracy vs SNR")
    plt.xlabel("SNR")
    plt.ylabel("Accuracy")
    plt.grid()
    plt.savefig(SavePath)
    plt.savefig(SavePath[:-3] + ".eps")
    plt.show()

Robust_CNN = tf.keras.models.load_model("Robust_CNN")
EvaluateData(Robust_CNN, Clean_X_Valid, Clean_y_Valid, "Robust_CNN_Accuracy.png")